In [2]:
import os
import sys
import tensorflow as tf

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from ml.models.three_d import voxnet

/home/amy/elvo-analysis/ml/models/three_d/voxnet.py:79: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(input_shape=(1, 32, 32..., kernel_size=(5, 5, 5), filters=32, strides=(2, 2, 2), padding="valid", data_format="channels_first", kernel_initializer="normal", kernel_regularizer=<keras.reg..., bias_regularizer=<keras.reg...)`
  b_regularizer=l2(0.001),


DEBUG:root:Layer1:Conv3D shape=(None, 32, 14, 14, 14)
DEBUG:root:Layer3:Conv3D shape=(None, 32, 12, 12, 12)
DEBUG:root:Layer4:MaxPool3D shape=(None, 32, 6, 6, 6)
DEBUG:root:Layer5:Flatten shape=(None, 6912)
DEBUG:root:Layer6:Dense shape=(None, 128)
DEBUG:root:Layer8:Dense shape=(None, 2)
INFO:root:Model compiled!


/home/amy/elvo-analysis/venv/lib/python3.6/site-packages/keras/activations.py:186: UserWarning: Do not pass a layer instance (such as LeakyReLU) as the activation argument of another layer. Instead, advanced activation layers should be used just like any other layer in a model.
  identifier=identifier.__class__.__name__))
/home/amy/elvo-analysis/ml/models/three_d/voxnet.py:87: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.3)`
  self._mdl.add(Dropout(p=0.3))
/home/amy/elvo-analysis/ml/models/three_d/voxnet.py:99: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(kernel_size=(3, 3, 3), filters=32, strides=(1, 1, 1), padding="valid", data_format="channels_first", kernel_initializer="normal", kernel_regularizer=<keras.reg..., bias_regularizer=<keras.reg...)`
  b_regularizer=l2(0.001),
/home/amy/elvo-analysis/ml/models/three_d/voxnet.py:110: UserWarning: Update your `MaxPooling3D` call to the Keras 2 API: `MaxPooling3D(pool_size=(2, 2, 2), stri

In [8]:
import os
import csv
import numpy as np
import random
from keras.preprocessing.image import ImageDataGenerator
from google.cloud import storage

BLACKLIST = []


class ChunkGenerator(object):
    def __init__(self, dims=(32, 32, 32), batch_size=16,
                 shuffle=True,
                 validation=False,
                 test=False, split_test=False,
                 split=0.2):
        self.dims = dims
        self.batch_size = batch_size
        self.validation = validation

        self.datagen = ImageDataGenerator(
            rotation_range=20,
            width_shift_range=0.1,
            height_shift_range=0.1,
            zoom_range=[1.0, 1.1],
        )

        # Access Google Cloud Storage
        gcs_client = storage.Client.from_service_account_json(
            '/home/amy/credentials/client_secret.json'
        )
        bucket = gcs_client.get_bucket('elvos')

        # Get label data from Google Cloud Storage
        blob = storage.Blob('augmented_annotated_labels.csv', bucket)
        blob.download_to_filename('/home/amy/elvo-analysis/augmented_annotated_labels.csv')
        prelim_label_data = {}
        with open('/home/amy/elvo-analysis/augmented_annotated_labels.csv', 'r') as pos_file:
            reader = csv.reader(pos_file, delimiter=',')
            for row in reader:
                if row[1] != 'Unnamed: 0':
                    prelim_label_data[row[1]] = int(row[2])

        # Get 12096 random negatives from the label data to feed into our generator
        negative_counter = 0
        negative_label_data = {}
        print("getting 12096 random negative labels")
        while negative_counter < 12000:
            id_, label = random.choice(list(prelim_label_data.items()))
            if label == 0:
                negative_label_data[id_] = label
                del prelim_label_data[id_]
                negative_counter += 1
        
        # Get all of the positives from the label data
        label_data = {}
        print('getting positive labels')
        for id_, label in list(prelim_label_data.items()):
            if label == 1:
                label_data[id_] = label
        
        print("negative label data length:" + str(len(negative_label_data)))
        print("positive label data length:" + str(len(label_data)))
        
        # Put pos and neg together into one dictionary
        label_data.update(negative_label_data)

        # Get positive chunks
        print('getting positive chunks')
        pos_blobs = bucket.list_blobs(prefix='chunk_data/normal/positive')
        files = []
        for blob in pos_blobs:
            file = blob.name

            # Check blacklist
            blacklisted = False
            for each in BLACKLIST:
                if each in file:
                    blacklisted = True
                    
            file_id = blob.name.split('/')[-1]
            file_id = file_id.split('.')[0]

            if not blacklisted:
                # Add all data augmentation methods
                files.append({
                    "name": file,
                })

        print("getting negative chunks")
        # Get negative chunks that were chosen to be in the labels
        neg_blobs = bucket.list_blobs(prefix='chunk_data/normal/negative')
        for blob in neg_blobs:
            file = blob.name

            # Check blacklist
            blacklisted = False
            for each in BLACKLIST:
                if each in file:
                    blacklisted = True

            file_id = blob.name.split('/')[-1]
            file_id = file_id.split('.')[0]

            if file_id in negative_label_data and not blacklisted:
                files.append({
                    "name": file,
                })

        # convert labels from dict to np array
        print('converting labels from dict to np array')
        labels = np.zeros(len(files))
        for i, file in enumerate(files):
            filename = file['name']
            filename = filename.split('/')[-1]
            filename = filename.split('.')[0]
            labels[i] = label_data[filename]

        # Take into account shuffling
        if shuffle:
            tmp = list(zip(files, labels))
            random.shuffle(tmp)
            files, labels = zip(*tmp)
            labels = np.array(labels)

        # Split based on validation
        print('splitting based on validation split')
        if validation:
            if split_test:
                files = files[:int(len(files) * split / 2)]
                labels = labels[:int(len(labels) * split / 2)]
            else:
                files = files[:int(len(files) * split)]
                labels = labels[:int(len(labels) * split)]
        elif test:
            if split_test:
                files = files[int(len(files) * split / 2):
                              int(len(files) * split)]
                labels = labels[int(len(labels) * split / 2):
                                int(len(labels) * split)]
            else:
                raise ValueError('must set split_test to True if test')
        else:
            files = files[int(len(files) * split):]
            labels = labels[int(len(labels) * split):]
        print(np.shape(files))
        print(np.shape(labels))
        print("Negatives: {}".format(np.count_nonzero(labels == 0)))
        print("Positives: {}".format(np.count_nonzero(labels)))
        self.files = files
        self.labels = labels
        self.bucket = bucket

    def generate(self):
        steps = self.get_steps_per_epoch()
        # print(steps)
        while True:
            for i in range(steps):
                # print(i)
                # print("D")
                x, y = self.__data_generation(i)
                yield x, y

    def get_steps_per_epoch(self):
        return len(self.files) // self.batch_size

    def __data_generation(self, i):
        bsz = self.batch_size
        files = self.files[i * bsz:(i + 1) * bsz]
        labels = self.labels[i * bsz:(i + 1) * bsz]
        images = []

        # for improperly formatted images
        incorrect_shape_imgs = 0
        
        # Download files to tmp/npy/
        for i, file in enumerate(files):
            file_id = file['name'].split('/')[-1]
            file_id = file_id.split('.')[0]
            print(file_id)
            blob = self.bucket.get_blob(file['name'])
            blob.download_to_filename('/home/amy/data/chunks/{}.npy'
                                      .format(file_id))
            img = np.load('/home/amy/data/chunks/{}.npy'.format(file_id))
            print(np.shape(img))
            if np.shape(img)==(32, 32, 32):
                img = np.expand_dims(img, axis=0)
                images.append(img)
                print(np.shape(img))
            else:
                incorrect_shape_imgs += 1
                print("adding image to incorrect_shape_imgs")
                
            print("images shape: " + str(len(images)))

        print("final images shape: " + str(np.shape(images)))
        print("number incorrectly formatted images: " 
                  + str(incorrect_shape_imgs))
        
        images = np.array(images)
        
        # print("Loaded entire batch.")
        # print(np.shape(images))
        return images, labels



In [9]:
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

In [10]:
train_gen = ChunkGenerator()
val_gen = ChunkGenerator(validation=True)

DEBUG:urllib3.util.retry:Converted retries value: 3 -> Retry(total=3, connect=None, read=None, redirect=None, status=None)
DEBUG:google.auth.transport.requests:Making request: POST https://accounts.google.com/o/oauth2/token
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): accounts.google.com:443
DEBUG:urllib3.connectionpool:https://accounts.google.com:443 "POST /o/oauth2/token HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.googleapis.com:443
DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage/v1/b/elvos?projection=noAcl HTTP/1.1" 200 338
DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /download/storage/v1/b/elvos/o/augmented_annotated_labels.csv?alt=media HTTP/1.1" 200 11851901
getting 12096 random negative labels
getting positive labels
negative label data length:12000
positive label data length:12600
getting positive chunks
DEBUG:urllib3.connectionpool:Resetting dropped connection: www.googl

DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage/v1/b/elvos/o?pageToken=CjJjaHVua19kYXRhL25vcm1hbC9uZWdhdGl2ZS8yRDJTUjREUlJSNENDU1FLNDU0Lm5weQ%3D%3D&projection=noAcl&prefix=chunk_data%2Fnormal%2Fnegative HTTP/1.1" 200 861274
DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage/v1/b/elvos/o?pageToken=CjJjaHVua19kYXRhL25vcm1hbC9uZWdhdGl2ZS8ySk1IQjRMRVQ5VU1FSktSMTUwLm5weQ%3D%3D&projection=noAcl&prefix=chunk_data%2Fnormal%2Fnegative HTTP/1.1" 200 861490
DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage/v1/b/elvos/o?pageToken=CjJjaHVua19kYXRhL25vcm1hbC9uZWdhdGl2ZS8yS01LWFIyRzFCTEQwQzJHNTAzLm5weQ%3D%3D&projection=noAcl&prefix=chunk_data%2Fnormal%2Fnegative HTTP/1.1" 200 861142
DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage/v1/b/elvos/o?pageToken=CjJjaHVua19kYXRhL25vcm1hbC9uZWdhdGl2ZS8yV0VBUzJPWU1ITTBRUDBNMjQ1Lm5weQ%3D%3D&projection=noAcl&prefix=chunk_data%2Fnormal%2Fnegative HTTP/1.1" 200 861997


DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage/v1/b/elvos/o?pageToken=CjJjaHVua19kYXRhL25vcm1hbC9uZWdhdGl2ZS82UVhTNTM1SEdDVDNRVTdaNTc4Lm5weQ%3D%3D&projection=noAcl&prefix=chunk_data%2Fnormal%2Fnegative HTTP/1.1" 200 861363
DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage/v1/b/elvos/o?pageToken=CjJjaHVua19kYXRhL25vcm1hbC9uZWdhdGl2ZS82U1VBUFBSVTBWN01FVThIMjM1Lm5weQ%3D%3D&projection=noAcl&prefix=chunk_data%2Fnormal%2Fnegative HTTP/1.1" 200 861516
DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage/v1/b/elvos/o?pageToken=CjJjaHVua19kYXRhL25vcm1hbC9uZWdhdGl2ZS82WEI0RkhXQVhEVlE3OU5LNzQzLm5weQ%3D%3D&projection=noAcl&prefix=chunk_data%2Fnormal%2Fnegative HTTP/1.1" 200 861132
DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage/v1/b/elvos/o?pageToken=CjJjaHVua19kYXRhL25vcm1hbC9uZWdhdGl2ZS83MklZUldORkZZN09GUEo3NTU0Lm5weQ%3D%3D&projection=noAcl&prefix=chunk_data%2Fnormal%2Fnegative HTTP/1.1" 200 861516


DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage/v1/b/elvos/o?pageToken=CjJjaHVua19kYXRhL25vcm1hbC9uZWdhdGl2ZS9BVks3STVYTFNDUldNNkZONzc0Lm5weQ%3D%3D&projection=noAcl&prefix=chunk_data%2Fnormal%2Fnegative HTTP/1.1" 200 861332
DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage/v1/b/elvos/o?pageToken=CjJjaHVua19kYXRhL25vcm1hbC9uZWdhdGl2ZS9CMk5JUUlVMFBXWTBIQU5UMTE3Lm5weQ%3D%3D&projection=noAcl&prefix=chunk_data%2Fnormal%2Fnegative HTTP/1.1" 200 861746
DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage/v1/b/elvos/o?pageToken=CjFjaHVua19kYXRhL25vcm1hbC9uZWdhdGl2ZS9CNERBVVdNRE5RQ1ZJRThXODMubnB5&projection=noAcl&prefix=chunk_data%2Fnormal%2Fnegative HTTP/1.1" 200 861418
DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage/v1/b/elvos/o?pageToken=CjFjaHVua19kYXRhL25vcm1hbC9uZWdhdGl2ZS9CQktRTlJGRTZRSERES1RQNjYubnB5&projection=noAcl&prefix=chunk_data%2Fnormal%2Fnegative HTTP/1.1" 200 861355
DEBUG:urllib3.co

DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage/v1/b/elvos/o?pageToken=CjJjaHVua19kYXRhL25vcm1hbC9uZWdhdGl2ZS9ETVFSNTAyWE80WjVEU005NDczLm5weQ%3D%3D&projection=noAcl&prefix=chunk_data%2Fnormal%2Fnegative HTTP/1.1" 200 861400
DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage/v1/b/elvos/o?pageToken=CjJjaHVua19kYXRhL25vcm1hbC9uZWdhdGl2ZS9ETk82Q1MzWU5HRk1VV1hMMzUzLm5weQ%3D%3D&projection=noAcl&prefix=chunk_data%2Fnormal%2Fnegative HTTP/1.1" 200 861482
DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage/v1/b/elvos/o?pageToken=CjJjaHVua19kYXRhL25vcm1hbC9uZWdhdGl2ZS9EUlQzRjRJVFdQQ0NCQjZUNDgyLm5weQ%3D%3D&projection=noAcl&prefix=chunk_data%2Fnormal%2Fnegative HTTP/1.1" 200 861385
DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage/v1/b/elvos/o?pageToken=CjJjaHVua19kYXRhL25vcm1hbC9uZWdhdGl2ZS9EVllLWDRYTUJIRDU3TVpKMzc1Lm5weQ%3D%3D&projection=noAcl&prefix=chunk_data%2Fnormal%2Fnegative HTTP/1.1" 200 861536


...

DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage/v1/b/elvos/o?pageToken=CjFjaHVua19kYXRhL25vcm1hbC9uZWdhdGl2ZS9PVElQMjVZT1JIQktKUFpNNjkubnB5&projection=noAcl&prefix=chunk_data%2Fnormal%2Fnegative HTTP/1.1" 200 862288
DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage/v1/b/elvos/o?pageToken=CjJjaHVua19kYXRhL25vcm1hbC9uZWdhdGl2ZS9PWFJLNEk2Q1RYQ0VOQVpWNzA3Lm5weQ%3D%3D&projection=noAcl&prefix=chunk_data%2Fnormal%2Fnegative HTTP/1.1" 200 865308
DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage/v1/b/elvos/o?pageToken=CjNjaHVua19kYXRhL25vcm1hbC9uZWdhdGl2ZS9QMFdIMUwwQkExWVpYT0dXMjY0MC5ucHk%3D&projection=noAcl&prefix=chunk_data%2Fnormal%2Fnegative HTTP/1.1" 200 863360
DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage/v1/b/elvos/o?pageToken=CjJjaHVua19kYXRhL25vcm1hbC9uZWdhdGl2ZS9QNEFJQjhKTURZNlJEUkFQMjk0Lm5weQ%3D%3D&projection=noAcl&prefix=chunk_data%2Fnormal%2Fnegative HTTP/1.1" 200 861452
DEBUG:urll

DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage/v1/b/elvos/o?pageToken=CjJjaHVua19kYXRhL25vcm1hbC9uZWdhdGl2ZS9STEYxQUZFMEcxUTk4QlM5MzMwLm5weQ%3D%3D&projection=noAcl&prefix=chunk_data%2Fnormal%2Fnegative HTTP/1.1" 200 861276
DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage/v1/b/elvos/o?pageToken=CjJjaHVua19kYXRhL25vcm1hbC9uZWdhdGl2ZS9STUlENFhFS1RWRVlDU0dLMTI1Lm5weQ%3D%3D&projection=noAcl&prefix=chunk_data%2Fnormal%2Fnegative HTTP/1.1" 200 861385
DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage/v1/b/elvos/o?pageToken=CjJjaHVua19kYXRhL25vcm1hbC9uZWdhdGl2ZS9ST0RWQUlKNkE2UDRENTRZNDM3Lm5weQ%3D%3D&projection=noAcl&prefix=chunk_data%2Fnormal%2Fnegative HTTP/1.1" 200 861536
DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage/v1/b/elvos/o?pageToken=CjJjaHVua19kYXRhL25vcm1hbC9uZWdhdGl2ZS9ST01IVURMVENPRVhVUUNBNTc4Lm5weQ%3D%3D&projection=noAcl&prefix=chunk_data%2Fnormal%2Fnegative HTTP/1.1" 200 861483


DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage/v1/b/elvos/o?pageToken=CjJjaHVua19kYXRhL25vcm1hbC9uZWdhdGl2ZS9UVUkwU0hNTzNJTzRQVDhaMjQ1Lm5weQ%3D%3D&projection=noAcl&prefix=chunk_data%2Fnormal%2Fnegative HTTP/1.1" 200 861498
DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage/v1/b/elvos/o?pageToken=CjJjaHVua19kYXRhL25vcm1hbC9uZWdhdGl2ZS9UVkNQNDMzVEtPVkJDUVRXNjE3Lm5weQ%3D%3D&projection=noAcl&prefix=chunk_data%2Fnormal%2Fnegative HTTP/1.1" 200 861492
DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage/v1/b/elvos/o?pageToken=CjJjaHVua19kYXRhL25vcm1hbC9uZWdhdGl2ZS9UWE9HUzVDUTZSWk1TVjNXMzIyLm5weQ%3D%3D&projection=noAcl&prefix=chunk_data%2Fnormal%2Fnegative HTTP/1.1" 200 861407
DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage/v1/b/elvos/o?pageToken=CjNjaHVua19kYXRhL25vcm1hbC9uZWdhdGl2ZS9VNVpVSzlHWUNBTTRGWUs0MTAxNC5ucHk%3D&projection=noAcl&prefix=chunk_data%2Fnormal%2Fnegative HTTP/1.1" 200 861908
DE

DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage/v1/b/elvos/o?pageToken=CjJjaHVua19kYXRhL25vcm1hbC9uZWdhdGl2ZS9XWlpIMU04MlZERFBGMVdYNDEwLm5weQ%3D%3D&projection=noAcl&prefix=chunk_data%2Fnormal%2Fnegative HTTP/1.1" 200 861436
DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage/v1/b/elvos/o?pageToken=CjJjaHVua19kYXRhL25vcm1hbC9uZWdhdGl2ZS9YMUUzTUEzNVBGUFFYTkI5NjAxLm5weQ%3D%3D&projection=noAcl&prefix=chunk_data%2Fnormal%2Fnegative HTTP/1.1" 200 861048
DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage/v1/b/elvos/o?pageToken=CjJjaHVua19kYXRhL25vcm1hbC9uZWdhdGl2ZS9YOE5LNUVFWUNHSk5MU0Y5NTIyLm5weQ%3D%3D&projection=noAcl&prefix=chunk_data%2Fnormal%2Fnegative HTTP/1.1" 200 861479
DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage/v1/b/elvos/o?pageToken=CjJjaHVua19kYXRhL25vcm1hbC9uZWdhdGl2ZS9YRE5FV1BUS0ZMQ1dXVVNWODgwLm5weQ%3D%3D&projection=noAcl&prefix=chunk_data%2Fnormal%2Fnegative HTTP/1.1" 200 861576


DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage/v1/b/elvos/o?pageToken=CjJjaHVua19kYXRhL25vcm1hbC9uZWdhdGl2ZS9aVEdaQkkyUVgySlFGWElHNTY5Lm5weQ%3D%3D&projection=noAcl&prefix=chunk_data%2Fnormal%2Fnegative HTTP/1.1" 200 861292
DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage/v1/b/elvos/o?pageToken=CjFjaHVua19kYXRhL25vcm1hbC9uZWdhdGl2ZS9aVVIySUtUVUMyS1FWQjJYNDkubnB5&projection=noAcl&prefix=chunk_data%2Fnormal%2Fnegative HTTP/1.1" 200 861528
DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage/v1/b/elvos/o?pageToken=CjJjaHVua19kYXRhL25vcm1hbC9uZWdhdGl2ZS9aWEpINzBDOFBCRVBFNzZLMjI5Lm5weQ%3D%3D&projection=noAcl&prefix=chunk_data%2Fnormal%2Fnegative HTTP/1.1" 200 861630
DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage/v1/b/elvos/o?pageToken=CjJjaHVua19kYXRhL25vcm1hbC9uZWdhdGl2ZS9aWldDV0lZMDFYWFAzNldMMTI1Lm5weQ%3D%3D&projection=noAcl&prefix=chunk_data%2Fnormal%2Fnegative HTTP/1.1" 200 810526
converti

In [40]:
model = voxnet.VoxNet(2, "chunk weights")

DEBUG:root:Layer1:Conv3D shape=(None, 32, 14, 14, 14)
DEBUG:root:Layer3:Conv3D shape=(None, 32, 12, 12, 12)
DEBUG:root:Layer4:MaxPool3D shape=(None, 32, 6, 6, 6)
DEBUG:root:Layer5:Flatten shape=(None, 6912)
DEBUG:root:Layer6:Dense shape=(None, 128)
DEBUG:root:Layer8:Dense shape=(None, 2)
INFO:root:Model compiled!


/home/amy/elvo-analysis/ml/models/three_d/voxnet.py:79: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(input_shape=(1, 32, 32..., kernel_size=(5, 5, 5), filters=32, strides=(2, 2, 2), padding="valid", data_format="channels_first", kernel_initializer="normal", kernel_regularizer=<keras.reg..., bias_regularizer=<keras.reg...)`
  b_regularizer=l2(0.001),
/home/amy/elvo-analysis/venv/lib/python3.6/site-packages/keras/activations.py:186: UserWarning: Do not pass a layer instance (such as LeakyReLU) as the activation argument of another layer. Instead, advanced activation layers should be used just like any other layer in a model.
  identifier=identifier.__class__.__name__))
/home/amy/elvo-analysis/ml/models/three_d/voxnet.py:87: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.3)`
  self._mdl.add(Dropout(p=0.3))
/home/amy/elvo-analysis/ml/models/three_d/voxnet.py:99: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(kernel_size

In [41]:
# Train model on dataset
# model.fit_generator(generator=train_gen.generate(),
#                    validation_data=val_gen.generate(),
#                    steps_per_epoch=training_generator.get_steps_per_epoch(),
#                    validation_steps=validation_generator.get_steps_per_epoch(),
#                    epochs = 15)


model.fit(generator = train_gen.generate(), 
          samples_per_epoch = train_gen.get_steps_per_epoch(),
          nb_epoch = 15, valid_generator = val_gen.generate(), 
          nb_valid_samples = val_gen.get_steps_per_epoch(), 
          verbosity = 2)

INFO:root:Start training


/home/amy/elvo-analysis/ml/models/three_d/voxnet.py:166: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  nb_val_samples=nb_valid_samples,
/home/amy/elvo-analysis/ml/models/three_d/voxnet.py:166: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(generator=<generator..., verbose=2, callbacks=[<keras.ca..., validation_data=<generator..., steps_per_epoch=1204, epochs=15, validation_steps=301)`
  nb_val_samples=nb_valid_samples,


MTWW42SPCGLHEDKY269_21Epoch 1/15

DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage/v1/b/elvos/o/chunk_data%2Fnormal%2Fpositive%2FMTWW42SPCGLHEDKY269_21.npy HTTP/1.1" 200 837
DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /download/storage/v1/b/elvos/o/chunk_data%2Fnormal%2Fpositive%2FMTWW42SPCGLHEDKY269_21.npy?generation=1531338697875725&alt=media HTTP/1.1" 200 65664
(32, 32, 32)
(1, 32, 32, 32)
images shape: 1
WPNYHBQRGJ4NHU3C356_20
DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage/v1/b/elvos/o/chunk_data%2Fnormal%2Fpositive%2FWPNYHBQRGJ4NHU3C356_20.npy HTTP/1.1" 200 837
DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /download/storage/v1/b/elvos/o/chunk_data%2Fnormal%2Fpositive%2FWPNYHBQRGJ4NHU3C356_20.npy?generation=1531339227920505&alt=media HTTP/1.1" 200 65664
(32, 32, 32)
(1, 32, 32, 32)
images shape: 2
VDO7JNWOEJ5KGJYJ213_4
DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage

DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /download/storage/v1/b/elvos/o/chunk_data%2Fnormal%2Fnegative%2F59PNXPS5R67U6WQP444.npy?generation=1531258752670450&alt=media HTTP/1.1" 200 65664
(32, 32, 32)
(1, 32, 32, 32)
images shape: 2
MKWC8L1HGWJWZYQ6613
DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage/v1/b/elvos/o/chunk_data%2Fnormal%2Fnegative%2FMKWC8L1HGWJWZYQ6613.npy HTTP/1.1" 200 825
DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /download/storage/v1/b/elvos/o/chunk_data%2Fnormal%2Fnegative%2FMKWC8L1HGWJWZYQ6613.npy?generation=1531289950704324&alt=media HTTP/1.1" 200 65664
(32, 32, 32)
(1, 32, 32, 32)
images shape: 3
WMMP0B4WE4H12UL5362
DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage/v1/b/elvos/o/chunk_data%2Fnormal%2Fnegative%2FWMMP0B4WE4H12UL5362.npy HTTP/1.1" 200 825
DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /download/storage/v1/b/elvos/o/chunk_data%2Fnormal%2Fnegati

ValueError: Error when checking target: expected activation_30 to have shape (2,) but got array with shape (1,)